### Тест на других признаков и алгоритмах

Здесь сразу отметим, что данный блокнот дополнительный, приведенные здесь варианты признаков и алгоритмов по итогам не являются более оптимальными, чем в блокноте "(2) Машинное обучение датасета" 

#### Random forest + дополнительные признаки

Здесь добавим в наш тренировочный набор признаки, такие как "Month", "DayofMonth",	"DayOfWeek", и WN из UniqueCarrier, по которому было показано существенное преобладание в сторону класса "N" в "(1) Анализ датасета Flight Delay"

Подготовим данные для обучения

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

data = pd.read_csv('flight_delays_train.csv')
print(data.shape)
data

(100000, 9)


,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [2]:
data['Month']=data['Month'].map(lambda x: str(x)[2:])
data['DayofMonth']=data['DayofMonth'].map(lambda x: str(x)[2:])
data['DayOfWeek']=data['DayOfWeek'].map(lambda x: str(x)[2:])
data['HourDepTime']=data['DepTime'].map(lambda x: str(x)[:-2])
data['HourDepTime'].replace(['25','24',''],[0,0,0],inplace=True)
data['MinutesDepTime']=data['DepTime'].map(lambda x: str(x)[-2:])
data['HourDepTime'] = data['HourDepTime'].astype(str)
data['MinutesDepTime'] = data['MinutesDepTime'].astype(str)
data['DepTimeCorrect'] = data['HourDepTime'] + data['MinutesDepTime']
data['DepTimeCorrect']=data['DepTimeCorrect'].astype(int)
data

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,HourDepTime,MinutesDepTime,DepTimeCorrect
0,8,21,7,1934,AA,ATL,DFW,732,N,19,34,1934
1,4,20,3,1548,US,PIT,MCO,834,N,15,48,1548
2,9,2,5,1422,XE,RDU,CLE,416,N,14,22,1422
3,11,25,6,1015,OO,DEN,MEM,872,N,10,15,1015
4,10,7,6,1828,WN,MDW,OMA,423,Y,18,28,1828
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,4,3,1618,OO,SFO,RDD,199,N,16,18,1618
99996,1,18,3,804,CO,EWR,DAB,884,N,8,04,804
99997,1,24,2,1901,NW,DTW,IAH,1076,N,19,01,1901
99998,4,27,4,1515,MQ,DFW,GGG,140,N,15,15,1515


In [4]:
y = data['dep_delayed_15min']
X = data.drop(['dep_delayed_15min','DepTime', 'MinutesDepTime', 'HourDepTime', 'Origin', 'Dest'], axis=1)
X

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Distance,DepTimeCorrect
0,8,21,7,AA,732,1934
1,4,20,3,US,834,1548
2,9,2,5,XE,416,1422
3,11,25,6,OO,872,1015
4,10,7,6,WN,423,1828
...,...,...,...,...,...,...
99995,5,4,3,OO,199,1618
99996,1,18,3,CO,884,804
99997,1,24,2,NW,1076,1901
99998,4,27,4,MQ,140,1515


In [5]:
for index in range(0,len(X['UniqueCarrier'])):
    if (X['UniqueCarrier'][index] != 'WN'):
        X['UniqueCarrier'][index] = np.NaN
X

<ipython-input-5-6ab45e75c8b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['UniqueCarrier'][index] = np.NaN


,Month,DayofMonth,DayOfWeek,UniqueCarrier,Distance,DepTimeCorrect
0,8,21,7,NaN,732,1934
1,4,20,3,NaN,834,1548
2,9,2,5,NaN,416,1422
3,11,25,6,NaN,872,1015
4,10,7,6,WN,423,1828
...,...,...,...,...,...,...
99995,5,4,3,NaN,199,1618
99996,1,18,3,NaN,884,804
99997,1,24,2,NaN,1076,1901
99998,4,27,4,NaN,140,1515


In [6]:
uniquecarrier = X['UniqueCarrier']
uniquecarrier = pd.get_dummies(uniquecarrier)

In [7]:
X = pd.concat((X,uniquecarrier), axis=1)
X = X.drop(['UniqueCarrier'], axis=1)
X

,Month,DayofMonth,DayOfWeek,Distance,DepTimeCorrect,WN
0,8,21,7,732,1934,0
1,4,20,3,834,1548,0
2,9,2,5,416,1422,0
3,11,25,6,872,1015,0
4,10,7,6,423,1828,1
...,...,...,...,...,...,...
99995,5,4,3,199,1618,0
99996,1,18,3,884,804,0
99997,1,24,2,1076,1901,0
99998,4,27,4,140,1515,0


Теперь переходим к обучению

In [8]:
import sklearn
from sklearn.model_selection import train_test_split

# Разделяем на тренировочную выборку и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=11)
forest_params = {'criterion':['gini','entropy'],'max_depth': range(1,10), 'max_features': range(1,7)}
forest_grid = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, verbose=True)
forest_grid.fit(X_train, y_train)
print('Best cross-validation parameters:',forest_grid.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best cross-validation parameters: {'criterion': 'gini', 'max_depth': 8, 'max_features': 4}


In [11]:
forest = RandomForestClassifier(criterion='gini', max_depth=8, max_features = 4, n_estimators=100, n_jobs=-1, random_state=11)
forest.fit(X_train, y_train)

y_test_predict_forest = forest.predict(X_test)


In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print('(1) FOREST')
print()

# Print Confusion matrix:
print('Confusion matrix:')
cmatrix = confusion_matrix(y_test, y_test_predict_forest)
print(cmatrix)
print()

# Print classification_report:
print('Classification_report:')
print(classification_report(y_test, y_test_predict_forest))

(1) FOREST

Confusion matrix:
[[24094   194]
 [ 5372   340]]

Classification_report:
              precision    recall  f1-score   support

           N       0.82      0.99      0.90     24288
           Y       0.64      0.06      0.11      5712

    accuracy                           0.81     30000
   macro avg       0.73      0.53      0.50     30000
weighted avg       0.78      0.81      0.75     30000



In [13]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, forest.predict_proba(X_test)[:,1])

0.7132082097290499

Видим, что AUC-score увеличился, но не значительно и на реальных тестовых данных может не показать прироста, учитывая, что мы не наблюдали корреляцию по "Month", "DayofMonth",	"DayOfWeek" и Минутам отправления рейса.

Если пройтись по существенным значениям "Origin" и "Dest" и аналогично оставить только их для обучения, то существенного роста AUC-score мы не увидим

#### Decision Tree

Decision Tree посмотрим на 1 признаке для сравнения с Random forest на одном признаке

In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

data = pd.read_csv('flight_delays_train.csv')
print(data.shape)
data

(100000, 9)


,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [16]:
data['HourDepTime']=data['DepTime'].map(lambda x: str(x)[:-2])
data['HourDepTime'].replace(['25','24',''],[0,0,0],inplace=True)
data['HourDepTime']=data['HourDepTime'].astype(int)
data

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,HourDepTime
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,19
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,15
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,14
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,10
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,18
...,...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N,16
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N,8
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N,19
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N,15


In [17]:
y = data['dep_delayed_15min']
X = data.drop(['Month','DayofMonth','UniqueCarrier','Distance','DayOfWeek','dep_delayed_15min','DepTime', 'Origin', 'Dest'], axis=1)
X

,HourDepTime
0,19
1,15
2,14
3,10
4,18
...,...
99995,16
99996,8
99997,19
99998,15


In [18]:
import sklearn
from sklearn.model_selection import train_test_split

# Разделяем на тренировочную выборку и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

tree_params = {'criterion':['gini','entropy'],'max_depth': range(1,10),'max_features': range(1,2)}
dtc = DecisionTreeClassifier(random_state=11)
tree_grid = GridSearchCV(dtc, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(X_train, y_train)
print('Best cross-validation parameters:',tree_grid.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best cross-validation parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': 1}


In [20]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='gini', max_depth=3, max_features = 1, random_state=11)
dtc.fit(X_train, y_train)

y_test_predict = dtc.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print('(2) TREE DECISION')
print()

# Print Confusion matrix:
print('Confusion matrix:')
cmatrix = confusion_matrix(y_test, y_test_predict)
print(cmatrix)
print()

# Print classification_report:
print('Classification_report:')
print(classification_report(y_test, y_test_predict))

(2) TREE DECISION

Confusion matrix:
[[24238    50]
 [ 5644    68]]

Classification_report:
              precision    recall  f1-score   support

           N       0.81      1.00      0.89     24288
           Y       0.58      0.01      0.02      5712

    accuracy                           0.81     30000
   macro avg       0.69      0.50      0.46     30000
weighted avg       0.77      0.81      0.73     30000



In [22]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, dtc.predict_proba(X_test)[:,1])

0.6863223642965092

Видим, что AUC-score меньше, чем на алгоритме Random Forest

#### KNeighbors 

Аналогично посмотрим на алгоритме KNeighborsClassifier

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

data = pd.read_csv('flight_delays_train.csv')

data['HourDepTime']=data['DepTime'].map(lambda x: str(x)[:-2])
data['HourDepTime'].replace(['25','24',''],[0,0,0],inplace=True)
data['HourDepTime']=data['HourDepTime'].astype(int)

y = data['dep_delayed_15min']
X = data.drop(['Month','DayofMonth','UniqueCarrier','Distance','DayOfWeek','dep_delayed_15min','DepTime', 'Origin', 'Dest'], axis=1)
X

,HourDepTime
0,19
1,15
2,14
3,10
4,18
...,...
99995,16
99996,8
99997,19
99998,15


In [25]:
import sklearn
from sklearn.model_selection import train_test_split

# Разделяем на тренировочную выборку и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

In [27]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score
n_neighbors_array = [1, 3, 5, 7, 10, 15, 20, 100]
knn= KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid={'n_neighbors': n_neighbors_array})
grid.fit(X_train, y_train)

print('KNN NEIGHBORS')
print()

print ('Наилучший параметр кросс-валидации: ',grid.best_estimator_.n_neighbors)

KNN NEIGHBORS

Наилучший параметр кросс-валидации:  100


In [28]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

y_train_predict_knn = knn.predict(X_train)
y_test_predict_knn = knn.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print('Confusion matrix:')
cmatrix_knn = confusion_matrix(y_test, y_test_predict_knn)
print(cmatrix_knn)

# Вывод classification_report:
print('Classification_report:')
print(classification_report(y_test, y_test_predict_knn))

Confusion matrix:
[[24277    11]
 [ 5698    14]]
Classification_report:
              precision    recall  f1-score   support

           N       0.81      1.00      0.89     24288
           Y       0.56      0.00      0.00      5712

    accuracy                           0.81     30000
   macro avg       0.68      0.50      0.45     30000
weighted avg       0.76      0.81      0.73     30000



In [30]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, knn.predict_proba(X_test)[:,1])

0.6853800690442514

Видим, что AUC-score меньше, чем на алгоритме Random Forest